In [66]:
import cPickle as pickle
import pandas as pd, numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

In [67]:
train = pd.read_csv('./final/train_800.csv')
test = pd.read_csv('./test.csv')
subm = pd.read_csv('./sample_submission.csv')



In [68]:
test.drop(test.index, inplace=True)

In [69]:
test.head()

,id,comment_text


In [70]:
train.count()

Unnamed: 0       1068
comment_text     1068
id               1068
identity_hate    1068
insult           1068
obscene          1068
severe_toxic     1068
threat           1068
toxic            1068
dtype: int64

In [88]:
test = pd.DataFrame([['1', 'fuck hisham'],
                     ['2', 'Ill kill her'],
                     ['3', 'I hate her'],
                     ['4', 'I\'ll blow up my school'],
                     ['5','My teacher hates me, never listens to my reasoning'],
                     ['6','I feel like murdering her'],
                     ['7','hey dont say that'],
                     ['8','what am I gonna do about her?'],
                     ['9','idk'],
                     ['10','I\'ll probably murder her']
                    ], columns=['id', 'comment_text'])

In [108]:
test = pd.DataFrame( [["user1", "Listen where are you?"],
          ["user2", "I am in the opposite building"] ,
         ["user1",  "Okay meet me in the foyer after 10mins and we’ll grab Ali and take him in the football ground"],
         ["user2", "Do you have something to attack him?"] ,
         ["user1",  "Yes I have a blade in my bag"]], columns=['id', 'comment_text'])

In [127]:
test = pd.DataFrame( [["user1", "Hide the knife somewhere before somebody sees it"],
          ["user2", "Do not worry I have wrapped it and hid in my bag"] ,
         ["user1",  "We will throw the knife once we get out of here"],
         ["user2", "yes for sure"]] 
                      , columns=['id', 'comment_text'])

In [154]:
test = pd.DataFrame( [["user1", "What are you doing now?"],
          ["user2", "I am playing pool with my friends at a pool hall."] ,
         ["user1",  "I didn't know you play pool. Are you having fun?"],
         ["user2", "I'm having a great time. How about you? What are you doing?"],
        ["user1", "I'm glad I'm not in your shoes."],
        ["user2","I'm taking a break from my homework. There seems to be no end to the amount of work I have to do."]] 
                      , columns=['id', 'comment_text'])

In [172]:
test = pd.DataFrame( [["user1", "I not in a mood. fuck off"],
          ["user2", "How to make a bomb"] ,
         ["user1",  "I hate you"],
         ["user2", "I dont hate you"]] 
                      , columns=['id', 'comment_text'])

In [173]:
test

,id,comment_text
0,user1,I not in a mood. fuck off
1,user2,How to make a bomb
2,user1,I hate you
3,user2,I dont hate you


In [174]:
label_cols = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']
train['none'] = 1-train[label_cols].max(axis=1)
train.describe()

,Unnamed: 0,identity_hate,insult,obscene,severe_toxic,threat,toxic,none
count,1068.000000,1068.000000,1068.000000,1068.000000,1068.000000,1068.000000,1068.000000,1068.000000
mean,533.500000,0.106742,0.261236,0.205993,0.098315,0.131086,0.487828,0.478464
std,308.449348,0.308929,0.439514,0.404614,0.297879,0.337653,0.500086,0.499770
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,266.750000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,533.500000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,800.250000,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,1.000000
max,1067.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [175]:
len(train),len(test)

(1068, 4)

In [176]:
COMMENT = 'comment_text'
train[COMMENT].fillna("unknown", inplace=True)
test[COMMENT].fillna("unknown", inplace=True)

In [177]:
import re,string
re_tok = re.compile(re.escape(string.punctuation))

In [178]:
def tokenize(s): return re_tok.sub(r' \1 ', s).split()

In [179]:
n = train.shape[0]
vec = TfidfVectorizer(ngram_range=(1,2), tokenizer=tokenize,
               min_df=3, max_df=0.9, strip_accents='unicode', use_idf=1,
               smooth_idf=1, sublinear_tf=1 )
trn_term_doc = vec.fit_transform(train[COMMENT])
with open('/home/ammar/study/8th semester/FYP/toxic comments/final/800/vec','wb') as fp:
        pickle.dump(vec,fp)
test_term_doc = vec.transform(test[COMMENT])

In [180]:
def pr(y_i, y):
    p = x[y==y_i].sum(0)
    return (p+1) / ((y==y_i).sum()+1)
x = trn_term_doc
test_x = test_term_doc

In [181]:
def get_mdl(y):
    y = y.values
    r = np.log(pr(1,y) / pr(0,y))
    m = LogisticRegression(C=4, dual=True)
    models = {}
    x_nb = x.multiply(r)
    result = m.fit(x_nb, y)
    return result, r,m
get_mdl.counter = 0

In [182]:
preds = np.zeros((len(test), len(label_cols)))
models = {}
# print(test_x)

In [183]:
for i, j in enumerate(label_cols):
    print('fit', j)
    m,r,model = get_mdl(train[j])
    print r
    with open('/home/ammar/study/8th semester/FYP/toxic comments/final/800/model'+str(i),'wb') as fp:
        pickle.dump(m,fp)
    with open('/home/ammar/study/8th semester/FYP/toxic comments/final/800/r'+str(i),'wb') as fp:
        pickle.dump(r,fp)
    preds[:,i] = m.predict_proba(test_x.multiply(r))[:,1]

('fit', 'toxic')
[[ 0.30596094  0.04373493 -0.94459451 ... -0.09589872 -0.16418971
  -0.09589872]]
('fit', 'severe_toxic')
[[ 1.49276978  2.00255521 -0.73558192 ...  1.88591541  1.89641242
   1.88591541]]
('fit', 'obscene')
[[ 1.2141917   1.14079949 -0.45841346 ...  1.0241597   1.03465671
   1.0241597 ]]
('fit', 'threat')
[[ 1.17046643  1.68025186 -1.1615452  ...  1.56361206  1.57410907
   1.56361206]]
('fit', 'insult')
[[ 0.90553856  0.83214635 -0.68018592 ...  0.71550656  0.72600356
   0.71550656]]
('fit', 'identity_hate')
[[ 1.40189679  1.91168222 -0.61279532 ...  1.79504243  1.80553943
   1.79504243]]


In [184]:
print r.shape
print m
print test_x.shape

(1, 6306)
LogisticRegression(C=4, class_weight=None, dual=True, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)
(4, 6306)


In [185]:
submid = pd.DataFrame({'id': test["id"]})
submission = pd.concat([submid, pd.DataFrame(preds, columns = label_cols)], axis=1)
# submission.to_csv('submission.csv', index=False)

In [186]:
train.count()

Unnamed: 0       1068
comment_text     1068
id               1068
identity_hate    1068
insult           1068
obscene          1068
severe_toxic     1068
threat           1068
toxic            1068
none             1068
dtype: int64

In [187]:
print(models)

{}


In [188]:
test

,id,comment_text
0,user1,I not in a mood. fuck off
1,user2,How to make a bomb
2,user1,I hate you
3,user2,I dont hate you


In [189]:
submission

,id,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,user1,0.953631,0.032620,0.992937,0.023591,0.877892,0.026077
1,user2,0.419519,0.070428,0.043518,0.097448,0.086648,0.035208
2,user1,0.941594,0.044028,0.063013,0.038816,0.890828,0.615524
3,user2,0.589443,0.047223,0.092089,0.031744,0.163485,0.189962


In [25]:
len(submission)

NameError: name 'submission' is not defined

In [42]:
# submission.T.apply()

<bound method DataFrame.sum of                        0          1          2          3          4  \
id                     1          2          3          4          5   
toxic           0.998568   0.991953   0.915644   0.689392   0.620449   
severe_toxic    0.193351   0.029783  0.0647688  0.0264427  0.0243453   
obscene          0.99999  0.0977326   0.081977   0.390622    0.15328   
threat         0.0919825   0.914922   0.247835   0.125681  0.0947823   
insult          0.999295   0.125326   0.847492   0.574722   0.356041   
identity_hate  0.0494893  0.0539898   0.630397   0.142122    0.12413   

                       5          6          7          8          9  
id                     6          7          8          9         10  
toxic           0.640229    0.67323   0.661768   0.609536   0.948162  
severe_toxic    0.242806  0.0434997  0.0256162   0.031362  0.0260644  
obscene         0.385169   0.484935  0.0567138  0.0584597   0.375432  
threat         0.0695505  0.0404911  

In [222]:
train = pd.read_csv('./train.csv')

In [223]:
train.tail()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
159566,ffe987279560d7ff,""":::::And for the second time of asking, when ...",0,0,0,0,0,0
159567,ffea4adeee384e90,You should be ashamed of yourself \n\nThat is ...,0,0,0,0,0,0
159568,ffee36eab5c267c9,"Spitzer \n\nUmm, theres no actual article for ...",0,0,0,0,0,0
159569,fff125370e4aaaf3,And it looks like it was actually you who put ...,0,0,0,0,0,0
159570,fff46fc426af1f9a,"""\nAnd ... I really don't think you understand...",0,0,0,0,0,0


In [224]:
train_plain =  train[(train['toxic'] == 0) & (train['severe_toxic'] == 0) & (train['obscene'] == 0) & (train['threat'] == 0) 
                     & (train['insult'] == 0) & (train['identity_hate'] == 0)]

In [225]:
train_plain.count()

id               143346
comment_text     143346
toxic            143346
severe_toxic     143346
obscene          143346
threat           143346
insult           143346
identity_hate    143346
dtype: int64

In [233]:
murder = train_plain[train_plain['comment_text'].apply(lambda x: 'murder' in x)]
shoot = train_plain[train_plain['comment_text'].apply(lambda x: 'shoot' in x)]
kill = train_plain[train_plain['comment_text'].apply(lambda x: 'kill' in x)]
bomb = train_plain[train_plain['comment_text'].apply(lambda x: 'bomb' in x)]

In [235]:
print murder.count()
print shoot.count()
print kill.count()
print bomb.count()

id               494
comment_text     494
toxic            494
severe_toxic     494
obscene          494
threat           494
insult           494
identity_hate    494
dtype: int64
id               322
comment_text     322
toxic            322
severe_toxic     322
obscene          322
threat           322
insult           322
identity_hate    322
dtype: int64
id               1476
comment_text     1476
toxic            1476
severe_toxic     1476
obscene          1476
threat           1476
insult           1476
identity_hate    1476
dtype: int64
id               335
comment_text     335
toxic            335
severe_toxic     335
obscene          335
threat           335
insult           335
identity_hate    335
dtype: int64


In [ ]:
murder_50 = murder.sample(n=50)
murder_50.count()

In [135]:
sample_300 = train_plain.sample(n=300)
sample_300.count()

id               300
comment_text     300
toxic            300
severe_toxic     300
obscene          300
threat           300
insult           300
identity_hate    300
dtype: int64

In [140]:
toxic_plain = pd.concat([train1,sample_300])
toxic_plain_shuffled = toxic_plain.sample(frac =1).reset_index(drop = True)
toxic_plain_shuffled.count()

id               408
comment_text     408
toxic            408
severe_toxic     408
obscene          408
threat           408
insult           408
identity_hate    408
dtype: int64

In [142]:
toxic_plain_shuffled.to_csv('/home/ammar/study/8th semester/FYP/toxic comments/train_created1')